In [1]:
from tweepy.streaming import StreamListener
from tweepy.auth import OAuthHandler
from tweepy import Stream

from datetime import datetime
import json

import pandas as pd
from pymongo import MongoClient

In [2]:
#pyenv
import os
from dotenv import load_dotenv
load_dotenv()
CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.environ.get('ACCESS_TOKEN_SECRET')

In [3]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#api = tweepy.API(auth)
#api.update_status('tweepy + oauth!')

In [17]:
client = MongoClient('localhost', 27017)
db = client.twitterdb
col = db.tweets
raw_col = db.raw_data
keywords = ['bieber']

In [30]:
class MyListener(StreamListener):
    def on_data(self,data):
        
        tweet = json.loads(data)
        created_at = tweet["created_at"]
        id_str = tweet[ "id_str"]
        text = tweet[ "text"]
        obj = {"created_at":created_at, "id_str":id_str, "text":text}
        print(obj)
        print('====================')
        tweetind = col.insert_one(obj).inserted_id
        print(data)
        #tweetind_2 = raw_col.insert_one( json.loads(json.dumps(tweet))  ).inserted_id
        return True
    
    def on_error(self, status):
        print(status)
        return False

In [31]:
mylistener = MyListener()
mystream = Stream(auth, listener = mylistener)

In [32]:
print('start stream')
#mystream.filter(track=keywords)
mystream.filter(track = keywords)
print('close stream')

start stream
{'created_at': 'Sat May 04 04:20:39 +0000 2019', 'id_str': '1124529246284713990', 'text': 'Cocaine’s hell of a drug'}
{"created_at":"Sat May 04 04:20:39 +0000 2019","id":1124529246284713990,"id_str":"1124529246284713990","text":"Cocaine\u2019s hell of a drug","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":178512025,"id_str":"178512025","name":"JG","screen_name":"18major","location":"Atlanta,Ga ","url":"https:\/\/medium.com\/@18major?source=linkShare-5ac3b9f5b12-1481860585.com.com.com.com","description":"Student at Georgia state(Public Policy- Major\/Econ-Minor) Typos are ever present","translator_type":"none","protected":false,"verified":false,"followers_count":2609,"friends_count":1005,"listed_count":33,"favourites_cou

{'created_at': 'Sat May 04 04:20:40 +0000 2019', 'id_str': '1124529251322142720', 'text': 'RT @klausheir: justin bieber foi a maior decepcao da minha vida me doi muito ter que cancelar ele pq sempre amei eel tanto bicho anos sendo…'}
{"created_at":"Sat May 04 04:20:40 +0000 2019","id":1124529251322142720,"id_str":"1124529251322142720","text":"RT @klausheir: justin bieber foi a maior decepcao da minha vida me doi muito ter que cancelar ele pq sempre amei eel tanto bicho anos sendo\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1862145900,"id_str":"1862145900","name":"trix","screen_name":"blckwisdow","location":"Rio de Janeiro, Brasil","url":"https:\/\/stars-sadness.tumblr.com","description":"you should shut the fuck up right

{'created_at': 'Sat May 04 04:20:41 +0000 2019', 'id_str': '1124529252668465152', 'text': 'RT @theshawnsgirl: Eu sou fã do Justin Bieber desde 2009, mas sou mulher e como mulher, na verdade como ser humano, não posso apoiar essa m…'}
{"created_at":"Sat May 04 04:20:41 +0000 2019","id":1124529252668465152,"id_str":"1124529252668465152","text":"RT @theshawnsgirl: Eu sou f\u00e3 do Justin Bieber desde 2009, mas sou mulher e como mulher, na verdade como ser humano, n\u00e3o posso apoiar essa m\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":813219317668052992,"id_str":"813219317668052992","name":"blondeau.","screen_name":"alicefofao","location":"\ud835\udcc8\ud835\udcb6\ud835\udcc2\ud835\udcc7\ud835\udcc1 \u2661","url":"https:\/\/

{'created_at': 'Sat May 04 04:20:41 +0000 2019', 'id_str': '1124529255143149574', 'text': 'JJSDNXJJDJS tea'}
{"created_at":"Sat May 04 04:20:41 +0000 2019","id":1124529255143149574,"id_str":"1124529255143149574","text":"JJSDNXJJDJS tea","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2823162551,"id_str":"2823162551","name":"u_u","screen_name":"piensoentumira","location":null,"url":null,"description":"de c\u00e1ncer como duki","translator_type":"regular","protected":false,"verified":false,"followers_count":4684,"friends_count":46,"listed_count":36,"favourites_count":33079,"statuses_count":79974,"created_at":"Sat Oct 11 17:14:56 +0000 2014","utc_offset":null,"time_zone":null,"geo_enabled":true,"lang":"en","contributors_enabled":false,"

{'created_at': 'Sat May 04 04:20:43 +0000 2019', 'id_str': '1124529260662800384', 'text': 'RT @leoneltudo: justin bieber desde que lo dejo selena gómez se volvio medio imbécil me parece'}
{"created_at":"Sat May 04 04:20:43 +0000 2019","id":1124529260662800384,"id_str":"1124529260662800384","text":"RT @leoneltudo: justin bieber desde que lo dejo selena g\u00f3mez se volvio medio imb\u00e9cil me parece","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":146224555,"id_str":"146224555","name":"PAOLA\ud83e\udd40","screen_name":"paola_marinv","location":"M\u00e9xico","url":null,"description":"self-love ain't selfish, my love reigns supreme","translator_type":"none","protected":false,"verified":false,"followers_count":1205,"friends_count":1590

...

{'created_at': 'Sat May 04 04:20:47 +0000 2019', 'id_str': '1124529278027218945', 'text': 'RT @lajose: Justin Bieber defendiendo a Chris Brown diciendo que el cometio un error.\n\nLes dejo una foto del error https://t.co/DCqZKxi3PI'}
{"created_at":"Sat May 04 04:20:47 +0000 2019","id":1124529278027218945,"id_str":"1124529278027218945","text":"RT @lajose: Justin Bieber defendiendo a Chris Brown diciendo que el cometio un error.\n\nLes dejo una foto del error https:\/\/t.co\/DCqZKxi3PI","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":936381265011531776,"id_str":"936381265011531776","name":"A","screen_name":"vasquez_ana22","location":null,"url":null,"description":"Vive y deja vivir\u2728","translator_type":"none","protected":false,"ve

{'created_at': 'Sat May 04 04:20:48 +0000 2019', 'id_str': '1124529282041155584', 'text': 'RT @louandharrxy: para justin bieber esto fue “un error” https://t.co/SRTxNJuR73'}
{"created_at":"Sat May 04 04:20:48 +0000 2019","id":1124529282041155584,"id_str":"1124529282041155584","text":"RT @louandharrxy: para justin bieber esto fue \u201cun error\u201d https:\/\/t.co\/SRTxNJuR73","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":998441156,"id_str":"998441156","name":"bar","screen_name":"barbarakatrich","location":"C\u00f3rdoba, Argentina","url":"https:\/\/instagram.com\/barbarakatrich?utm_source=ig_profile_share&igshid=1ue64ksjlv5ef","description":"var\u00f3n muerto abono pa mi huerto","translator_type":"regular","protected":false,"veri

{'created_at': 'Sat May 04 04:20:48 +0000 2019', 'id_str': '1124529282686939136', 'text': 'RT @tayIucifer97: y’all are wasting time dragging taylor swift because of a couple of drums while male artists are getting away with sh*t l…'}
{"created_at":"Sat May 04 04:20:48 +0000 2019","id":1124529282686939136,"id_str":"1124529282686939136","text":"RT @tayIucifer97: y\u2019all are wasting time dragging taylor swift because of a couple of drums while male artists are getting away with sh*t l\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":995116997955354624,"id_str":"995116997955354624","name":"TS7 IS COMIMG \ud83c\udf34\ud83c\udf34\ud83c\udf34\ud83c\udf34\ud83c\udf34\ud83c\udf34\ud83c\udf34","screen_name":"SwiftieWithin","location

{"created_at":"Sat May 04 04:20:48 +0000 2019","id":1124529283659997184,"id_str":"1124529283659997184","text":"RT @holdtightlive: THIS IS WHAT WE DESERVE, WE\u2019VE STARVED FOR ALMOST FOUR YEARS, THE ONLY THING WE GOT WAS COLLABS. THIS IS OUR YEAR THANK\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/iphone\" rel=\"nofollow\"\u003eTwitter for iPhone\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":2438281604,"id_str":"2438281604","name":"\u2661","screen_name":"blancabizzle","location":"#blacklivesmatter","url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":954,"friends_count":1993,"listed_count":5,"favourites_count":42737,"statuses_count":23730,"created_at":"Fri Apr 11 10:15:00 +0000 2014","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":"en","contributors

KeyboardInterrupt: 

In [ ]:
class LimitedListener(StreamListener):

    def __init__(self):
        super().__init__()
        self.counter = 0
        self.limit = 10

    def on_status(self, status):
        try:
            userid = status.user.id_str
            geo = str(status.coordinates)
            if geo != "None":
                print(userid + ',' + geo)
            else:
                print("No coordinates")
            self.counter += 1
            if self.counter < self.limit:
                return True
            else:
                twitterStream.disconnect()
        except BaseException as e:
            print('failed on_status,',str(e))
            time.sleep(5)

In [ ]:
limited_listener = LimitedListener()
limited_stream = Stream(auth, listener = limited_listener)
print('inicia limited stream')
limited_stream.filter(track=keywords)
print('fecha limited stream')

In [ ]:
"""
client = MongoClient('localhost', 27017)
db_test = client.testdb
collection_test = db_test.col_test
obj_test = {"putetinho": "ayn ayn"}
collection_test.insert_one(obj_test)
"""

In [ ]:
col.find_one()

In [ ]:
dataset = [{"created_at":item["created_at"],
           "text":item["text"],
           } for item in col.find()]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [ ]:
word_count = pd.DataFrame(cv.get_feature_names(), columns=["word"])
word_count["count"] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values("count", ascending=False).reset_index(drop=True)
word_count[:50]